In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import warnings
# filter some warning messages
warnings.filterwarnings("ignore") 

####################you will need to change some paths here!#####################
#list of input files
filename_origin='f:/data/project_data/NASA_biophysical/collocated_data/NPPSD_GOA_allseabird_full_eddy_info.nc'
# output files
filename_origin_out='f:/data/project_data/NASA_biophysical/collocated_data/NPPSD_GOA_allseabird_full_eddy_info_envdata.nc'


In [ ]:
ds_bird = xr.open_dataset(filename_origin_out)
ds_bird


# collocate color

In [ ]:
ds_color = xr.open_dataset('https://rsg.pml.ac.uk/thredds/dodsC/CCI_ALL-v4.2-DAILY')
for var in ds_color:
    if not var=='chlor_a':
        ds_color = ds_color.drop(var)
ds_color

In [ ]:
ilen_bird = len(ds_bird.lat)
ds_data = ds_color
for var in ds_data:
    var_tem=var
    ds_bird[var_tem]=xr.DataArray(np.nan*np.empty(ilen_bird, dtype=str(ds_data[var].dtype)), coords={'index': ds_bird.index}, dims=('index'))
    ds_bird[var_tem].attrs=ds_data[var].attrs
print('var',var_tem)
for i in range(ilen_bird):
    t1,t2 = ds_bird.time64[i]-np.timedelta64(5,'D'), ds_bird.time64[i]+np.timedelta64(5,'D')
    lat1,lat2=ds_bird.lat[i]+.1,ds_bird.lat[i]-.1
    lon1,lon2=ds_bird.lon[i]-.1,ds_bird.lon[i]+.1
    tem = ds_data.sel(time=slice(t1,t2),lat=slice(lat1,lat2),lon=slice(lon1,lon2)).load()
    if len(tem)<1:
        continue
    tem = tem.mean(keep_attrs=True)
    for var in ds_data:
        var_tem=var
        ds_bird[var_tem][i]=tem[var].data
    if int(i/100)*100==i:
        print(i,len(ds_bird.lat))
#output data
df_bird = ds_bird.to_dataframe()
df_bird.to_csv(filename_bird_out)
ds_bird.to_netcdf(filename_bird_out_netcdf)